### Problem Context
We wish to compute the laziest way to dial given n-digit number on a standard push button telephone
(with 12 keys) using two fingers. We assume that the two fingers start out on the * and # keys, and that the
effort required to move a finger from one button to another is proportional to the Euclidean distance
between them (assume that the digit "1" is at position (0, 0), and the digit "5" at position (1,1)). Design an
algorithm in python that computes the method of dialing that involves moving your fingers the smallest
amount of total distance

The telephone keypad format that we have assumed looks like this:  
![Key pad Image](key_pad.png)

Let's assign cartesian coordinates to all keys on the telephone. We will store this as a dictionary of Tuples, where each tuple stores coordinates for that key in the form **(x,y)** For e.g. coordinates of "**1**" will be stored as **(0,0)**

In [ ]:
coordinates = {'1': (0,0),
              '2': (1,0),
              '3': (2,0),
              '4': (0,1),
              '5': (1,1),
              '6': (2,1),
              '7': (0,2),
              '8': (1,2),
              '9': (2,2),
              '*': (0,3),
              '0': (1,3),
              '#': (2,3)}

#### Creating function for computing Euclidean distance

In [ ]:
import math
from ast import literal_eval
def distance(key1: str, key2: str) -> float:
    """
    Returns the euclidean distance between two given keys on the push-button telephone
    Euclidean distance is computed as the pythagoras distance between two points on cartesian coordinate system
    """
    x1, y1 = coordinates.get(key1)
    x2, y2 = coordinates.get(key2)
    
    return math.sqrt((x1-x2)**2 + (y1-y2)**2)

## Designing the algorithm

##### Algorithm 1
A basic way to approach this problem is to be locally greedy and choose the finger resulting into a minimum distance for the next digit, based upon the current position of left and right fingers. For e.g. given we start at (\*, #) and the next digit to press is **1**, the distance of pressing it with left finger is **3** units, whereas for right finger it is **3.6**. Therfore, the locally greedy approach says we use the left finger to press this digit. After that, we continue in similar fashion for rest of the digits  
**Time Complexity**: O(n)
**Space Complexity**: O(1)

However, the above approach results into a higher total distance in cases where you can strategically choose a high distance finger for one digit and still end up with an overall lower total distance traversed. For e.g. If we are dialling **"583"**  
**Output based on algorithm 1**: (5.47, \[('\*','#'),('5','#'),('8','#'),('3','#')\]) 

Whereas, if we try minimizing the overall cost, we can create a different and better outcome:  
**Output based on overall cost minimization**: (5.06, \[('\*','#'),('5','#'),('5','8'),('3','8')\])

Based on the problem statement, we need to compute the smallest amount of total distance. Therefore, I have to design an algorithm to return **output 2** instead of **output 1**

##### Algorithm 2
An easiest way to minimize overall cost is to programatically generate all possible paths, compute the overall distance for each path, and choose the one with minimum distance. As there are two possible choices (left or right finger to dial) at every digit on the telephone number, the time complexity for this algorithm would be **O(n.2<sup>n</sup>)**  
**Time Complexity**: O(n.2<sup>n</sup>)  
**Space Complexity**: O(2<sup>n</sup>)

In [ ]:
# Let's write a function which returns the computed distance & finger positions for a given digit,
# positions of left and right finger, and which finger to use for dialling the digit
def compute_path(digit: str, key1: str, key2: str, finger: str):# -> tuple[float,list[tuple[str]]]
    """
    Returns the distance as float and path in form of list of tuples
    digit: the digit/key to be dialled
    key1: position of left finger
    key2: position of right finger
    finger: l/r (l for left, and r for right)
    For e.g.
    compute_path('4',('1','3'),'l') would return -> (1.0,[('4','3')])
    """
    if(finger=='l'):
        return (distance(key1, digit),(digit,key2))
    else:
        return (distance(key2, digit),(key1,digit))

In [ ]:
# Generating all possible combinations for a given string of length "n"
def generate_all_comb(set: set, n: int) -> list: 
    """
    This is a wrapper function calling the main recursive function to generate the possible combinations
    set: the set of different possible choices. In our case, it is ['l','r']
    n: the length of string
    """
    a = len(set)
    output = []
    generate_all_comb_rec(set, "", a, n, output)
    return output

def generate_all_comb_rec(set: set, prefix: str, a: int, n: int, output: list) -> None: 
    """
    This function recursively generates as possible combinations by adding each element of the set to the prefix
    and modifies the passed list output to contain those combinations
    set: the set of different possible choices
    prefix: what to add to each element of set
    a: length of the set
    n: length on string
    output: list to be modified. After function call, all possible combinations would be appended to this list
    """
    # Base case: n is 0, 
    # return prefix 
    if (n == 0) : 
        return output.append(prefix)
         
    for i in range(a): 
  
        # Next character of input added 
        newPrefix = prefix + set[i] 
          
        # n is decreased, because  
        # we have added a new character 
        generate_all_comb_rec(set, newPrefix, a, n - 1, output)

In [ ]:
def compute_laziest_path(telephone_number: str): # -> tuple[float,list[tuple[str]]]
    """
    This function takes a valid telephone number string as input and returns the minimum overall distance
    as well as the path taken to reach that minimum distance. It uses "Algorithm 2" explained above.
    """
    n = len(telephone_number)
    # Assigning a default result having higher cost than any path for a given string of length n
    result = (4*n,["invalid path"])
    # Initializing starting position
    start_pos = ('*','#')
    # Iterating through all 2^n combinations
    for comb in generate_all_comb(['l','r'],n):
        D = 0
        Path = [start_pos]
        pos = start_pos
        for i in range(n):
            # With each digit dialed, we store the results into step
            step = compute_path(telephone_number[i],*pos,comb[i])
            # Add the distance for the current digit dialled
            D += step[0]
            # Update the position of fingers
            pos = step[1]
            # Append the current position of fingers to the path
            Path.append(pos)
        # If the current combination distance is lower than the result distance, then update the result variable
        if(D < result[0]):
            result = (D, Path)
    
    return result

In [ ]:
compute_laziest_path('583')

##### Thinking Further
While this gives us the right answer, this is computationally expensive and if we look closely, we are repeating many calculations multiple times. For e.g. in two different combinations "llr" and "lll", we are essentially doing the same path and distance computations until the first two digits.

To optimize this, we can utilize dynamic programming and store partial paths & distance values to fetch later and use again. This would result into a higher space complexity but lower time complexity & computation time.

##### Algorithm 3
Cache paths & distances at the tree roots & use them at higher level in the branches to save up on computation time. Additionally, we can also pre-compute & save distances between any possible combination of two-keys to save computation time on distance calculation.  
**Time Complexity**: O(2<sup>n</sup>)  
**Space Complexity**: O(2<sup>n</sup>) (It also stores a KxK length dictionary which grows at O(k<sup>2</sup>) but that would become insignificant compared to the Paths dictionary growing at O(2<sup>n</sup>))

In [ ]:
# Creating an empty dictionary to store distance values
computed_dist = {}
# Looping through all keys to compute and add key combinations as dictionary key, and their distance as values
for key1 in coordinates.keys():
    for key2 in coordinates.keys():
        computed_dist[(key1,key2)] = distance(key1,key2)

In [ ]:
# Similar to the previous compute_path function, but this one does not compute the distances. Instead it just
# fetches them from a pre-computed dictionary
def compute_path_new(digit: str, key1: str, key2: str, finger: str):# -> tuple[float,list[tuple[str]]]
    """
    Returns the distance as float and path in form of list of tuples
    digit: the digit/key to be dialled
    key1: position of left finger
    key2: position of right finger
    finger: l/r (l for left, and r for right)
    For e.g.
    compute_path_new('4',('1','3'),'l') would return -> (1.0,[('4','3')])
    """
    if(finger=='l'):
        return (computed_dist.get((key1, digit)),(digit,key2))
    else:
        return (computed_dist.get((key2, digit)),(key1,digit))

In [ ]:
def compute_laziest_path_new(telephone_number: str): # -> tuple[float,list[tuple[str]]]
    """
    This function takes a valid telephone number string as input and returns the minimum overall distance
    as well as the path taken to reach that minimum distance, similar to function "compute_laziest_path".
    However, it uses "Algorithm 3" utilizing dynamic programming to re-use previously computed results.
    """
    # Initializing dictionary having path as associated cost. 
    # Initial path is starting finger positions with distance 0
    Path_dist = {"[('*','#')]": 0.0}
    # Iterating through every digit of the phone number
    for digit in telephone_number:
        # Creating an empty dictionary to store paths at every digit
        temp_dict = {}
        # Iterating through the paths created unitl previous digit and adding left & right branches
        for path in Path_dist.keys():
            path_till_here = literal_eval(path)
            dist_till_here = Path_dist.get(path)
            for side in ['l','r']:
                temp_result = compute_path_new(digit,*(path_till_here[-1]),side)
                new_path = path_till_here + [temp_result[1]]
                new_dist = dist_till_here + temp_result[0]
                temp_dict[str(new_path)] = new_dist
        Path_dist = temp_dict.copy()
    min_D = min(Path_dist.values())
    min_path = [literal_eval(key) for key in Path_dist.keys() if Path_dist[key]==min(Path_dist.values())]
    return (min_D,min_path[0])

In [ ]:
def main():
    trigger = 'Y'
    while(trigger!='N'):
        input_tel_num = input("Enter a valid number made up of 12 telephone keys: (Press 'N' to exit)\n")
        if(input_tel_num == 'N'):
            trigger = 'N'
        else:
            print(compute_laziest_path_new(input_tel_num))
            
if __name__ == "__main__":
    main()